# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kempsey,-31.0833,152.8333,6.08,87,13,1.34,AU,1686337471
1,1,walvis bay,-22.9575,14.5053,13.94,76,0,5.18,NaN,1686337628
2,2,sitka,57.0531,-135.3300,11.94,87,100,0.00,US,1686337628
3,3,pangody,65.8500,74.4859,4.28,59,4,2.69,RU,1686337629
4,4,iqaluit,63.7506,-68.5145,4.83,75,100,2.24,CA,1686337629
...,...,...,...,...,...,...,...,...,...,...
572,572,kiunga,-6.1219,141.2906,21.51,100,95,0.77,PG,1686337750
573,573,selizharovo,56.8519,33.4487,9.61,64,2,4.35,RU,1686337750
574,574,la sarre,48.8002,-79.1996,20.29,56,83,0.38,CA,1686337750
575,575,kristinehamn,59.3098,14.1081,18.31,41,3,3.03,SE,1686337750


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_data_map = city_data_df.hvplot.points(
                "Lng",
                "Lat",
                tiles = "EsriNatGeo",
                frame_width = 700,
                frame_height = 500,
                size = "Humidity",
                color = "City",
                geo = True
                )
    
    
# Display the map
# YOUR CODE HERE
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values

city_data_df = city_data_df.loc[(city_data_df['Max Temp']>=20) & (city_data_df['Max Temp']<=28)]
city_data_df = city_data_df.loc[(city_data_df['Wind Speed'] < 5)]
city_data_df = city_data_df.loc[(city_data_df['Cloudiness'] < 10)]

# Drop any rows with null values

city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,fillmore,43.6666,-92.1002,27.68,29,0,2.06,US,1686337638
61,61,quelimane,-17.8786,36.8883,25.97,83,0,3.60,MZ,1686337641
84,84,baoying,33.2292,119.3092,24.79,62,0,3.72,CN,1686337645
136,136,port douglas,-16.4833,145.4667,20.75,84,4,3.41,AU,1686337658
149,149,atherton,53.5237,-2.4935,21.05,49,6,2.24,GB,1686337660


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']]
hotel_df = hotel_df.dropna()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
50,fillmore,US,43.6666,-92.1002,29,
61,quelimane,MZ,-17.8786,36.8883,83,
84,baoying,CN,33.2292,119.3092,62,
136,port douglas,AU,-16.4833,145.4667,84,
149,atherton,GB,53.5237,-2.4935,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
params = {
        "radius":radius,
        "apiKey":geoapify_key,
        "categories": categories
          }
        

        

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    city = row['City']
    latitude = row['Lat']
    longitude =  row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["text"] = f"{city}"
    
    params['text'] = f"{city}"
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fillmore - nearest hotel: The Jail House Inn Bed & Breakfast
quelimane - nearest hotel: Mabassa
baoying - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
atherton - nearest hotel: Britannia Hotel
hamilton - nearest hotel: North Vista Manor
malaunay - nearest hotel: Campanile
alofi - nearest hotel: Matavai Resort
khovd - nearest hotel: Khovd hotel
colonia - nearest hotel: Wasserturm Hotel Cologne
peruibe - nearest hotel: Brisamar
kralupy nad vltavou - nearest hotel: Libuše
arraial do cabo - nearest hotel: No hotel found
tete - nearest hotel: Zambezi Hotel
ambovombe - nearest hotel: La source
tolanaro - nearest hotel: Hôtel Mahavokey
le port - nearest hotel: No hotel found
qom - nearest hotel: لاله
leava - nearest hotel: Fia fia
zandvoort - nearest hotel: Hotel Faber
imbituba - nearest hotel: Silvestre Praia Hotel
conceicao da barra - nearest hotel: Porto Marlin Hotel
abbeville - nearest hotel: Hôtel Mercure
morehead city - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
50,fillmore,US,43.6666,-92.1002,29,The Jail House Inn Bed & Breakfast
61,quelimane,MZ,-17.8786,36.8883,83,Mabassa
84,baoying,CN,33.2292,119.3092,62,No hotel found
136,port douglas,AU,-16.4833,145.4667,84,Mantra Aqueous Port Douglas
149,atherton,GB,53.5237,-2.4935,49,Britannia Hotel
154,hamilton,US,39.1834,-84.5333,29,North Vista Manor
162,malaunay,FR,49.5271,1.0429,44,Campanile
184,alofi,NU,-19.0595,-169.9187,83,Matavai Resort
207,khovd,MN,48.0056,91.6419,36,Khovd hotel
213,colonia,DE,50.9333,6.9500,44,Wasserturm Hotel Cologne


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_df_map = hotel_df.hvplot.points(
                "Lng",
                "Lat",
                tiles = "OSM",
                frame_width = 700,
                frame_height = 500,
                size = "Humidity",
                color = "City",
                geo = True,
                hover_cols = ["Hotel Name", "Country"]
                )
# Display the map
# YOUR CODE HERE
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)